任务：主 agent 接受来自用户的任务，首先确定问题是否能够回答（护栏边界内），如果在护栏内，那么调用对应的 agent 回答

In [ ]:
from agents.extensions.models.litellm_model import LitellmModel
from agents import set_tracing_disabled
import os
from dotenv import load_dotenv
from openai import AsyncOpenAI


# 加载环境变量
load_dotenv()
# 从环境变量中读取api_key
api_key = os.getenv('DEEPSEEK_API_KEY')
base_url = "https://api.deepseek.com/v1"
chat_model = "deepseek/deepseek-chat"

set_tracing_disabled(disabled=True)
llm = LitellmModel(model=chat_model, api_key=api_key, base_url=base_url)

In [ ]:
from agents import Agent
agent = Agent(
    name="Math Tutor",
    instructions="You provide help with math problems. Explain your reasoning at each step and include examples",
    model=llm,
)

In [ ]:
from agents import Agent

# 历史
history_tutor_agent = Agent(
    name="History Tutor",
    handoff_description="Specialist agent for historical questions",
    instructions="You provide assistance with historical queries. Explain important events and context clearly.",
    model=llm,
)

# 数学
math_tutor_agent = Agent(
    name="Math Tutor",
    handoff_description="Specialist agent for math questions",
    instructions="You provide help with math problems. Explain your reasoning at each step and include examples",
    model=llm,
)

In [ ]:
from agents import GuardrailFunctionOutput, InputGuardrail, Agent, Runner
from pydantic import BaseModel

class HomeworkOutput(BaseModel):
    is_homework: bool
    reasoning: str

guardrail_agent = Agent(
    name="Guardrail check",
    instructions="Check if the user is asking about homework.",
    output_type=HomeworkOutput,
    model=llm,
)

async def homework_guardrail(ctx, agent, input_data):
    result = await Runner.run(guardrail_agent, input_data, context=ctx.context)
    final_output = result.final_output_as(HomeworkOutput)
    return GuardrailFunctionOutput(
        output_info=final_output,
        tripwire_triggered=not final_output.is_homework,
    )

In [ ]:
# 基于用户问题，选择合适的 agent
# triage_agent = Agent(
#     name="Triage Agent",
#     instructions="You determine which agent to use based on the user's homework question",
#     handoffs=[history_tutor_agent, math_tutor_agent],
#     model=llm,
# )

triage_agent = Agent(
    name="Triage Agent",
    instructions="You determine which agent to use based on the user's homework question",
    # 允许代理将特定任务委派给其他代理
    handoffs=[history_tutor_agent, math_tutor_agent],
    # 可以对输入到代理的内容进行验证
    input_guardrails=[
        InputGuardrail(guardrail_function=homework_guardrail),
    ],
    model=llm,
)

In [ ]:
from agents import Runner
import asyncio

async def main():
    result = await Runner.run(triage_agent, "What is the capital of France?")
    print(result.final_output)


# async def main():
#     result = await Runner.run(triage_agent, "does drinking tee good for the body?")
#     print(result.final_output)

    
# if __name__ == "__main__":
    # asyncio.run(main())

await main()

In [ ]:
from agents import Runner
result = await Runner.run(triage_agent, "does drinking tee good for the body?")
print(result.final_output)

报错没有解决！

```
BadRequestError: litellm.BadRequestError: DeepseekException - Failed to deserialize the JSON body into the target type: response_format: This response_format type is unavailable now at line 1 column 560
```